# Regression homeweek in week 11

**info**

- SuJinhua
- 2017201620


In [166]:
import pandas as pd
df = pd.read_excel('table5-6.xlsx')
df = df.iloc[:,1:]

In [175]:
import numpy as np
from sklearn.feature_selection import f_regression
from scipy import stats
class gradual_regression(object):
    def __init__(self,df,t_test_reject_value):
        self.df = df
        self.reject = t_test_reject_value
    def _data(self,l):
        return self.df[l].to_numpy()
    def _fail_idx(self,p):
        for i in range(len(p)-1,-1,-1):
            if p[i] >= self.reject:
                yield i
    def gradual_reg(self):
        select_set = list(self.df.columns)
        y = self._data(select_set.pop())
        init_set = []
        garbage_set = []
        count = 0
        print('No.',count,init_set)
        while len(select_set) > 0:
            pop = select_set.pop()
            init_set.append(pop)
            count += 1
            print('No.',count,init_set)
            beta,t,F,p = self._regression(self._data(init_set),y)
            flag = 1 
            for i in self._fail_idx(p):
                flag = 0
                temp = init_set.pop(i)
                count += 1
                print('No.',count,init_set)
                garbage_set.append(temp)
                if pop == temp:
                    break
                flag = 1
            if flag == 1:
                select_set += garbage_set
                garbage_set = []
        beta,t,F,p = self._regression(self._data(init_set),y)
        return self.pretty_report(init_set,beta,t,F,p,'gradual')
    def backward_reg(self):
        init_set = list(self.df.columns)
        y = self._data(init_set.pop())
        count = 0
        print('No.',count,init_set)
        while True:
            beta,t,F,p = self._regression(self._data(init_set),y)
            flag = 0
            for i in self._fail_idx(p):
                init_set.pop(i)
                flag = 1
                break
            if flag == 1:
                count += 1
                print('No.',count,init_set)
                continue
            beta,t,F,p = self._regression(self._data(init_set),y)
            return self.pretty_report(init_set,beta,t,F,p,'backward')
    def pretty_report(self,init_set,beta,t,F,p,method):
        def linear(init_set,beta,t,F,p):
            init_set.insert(0,'cons_')
            F = list(F)
            p = list(p)
            F.insert(0,0)
            p.insert(0,0)
            s = ''
            for i in range(len(init_set)):
                s += ' | '.join([init_set[i],'%.2f' % beta[i],'%.2f' % t[i],'%.2f' % F[i],'%.2f' % p[i]]) + '\n'
            return s
        s = \
"""
regression result\n
method: {method}\n
-----------------\n
y | beta | t | F_i | p(F)\n
{linear_com}
"""
        return s.format(method = method,linear_com = linear(init_set,beta,t,F,p))
    def _regression(self,x,y):
        F_ = []
        for i in range(x.shape[1]):
            if i != x.shape[1] - 1:
                x_ = np.hstack((np.ones((x.shape[0],1)),x[:,:i],x[:,i+1:]))
            else:
                x_ = np.hstack((np.ones((x.shape[0],1)),x[:,:i]))
            beta = np.linalg.inv(x_.T.dot(x_)).dot(x_.T.dot(y))
            SSE = np.sum((y - x_.dot(beta))**2)
            F_.append(SSE)
        x = np.hstack((np.ones((x.shape[0],1)),x))
        beta = np.linalg.inv(x.T.dot(x)).dot(x.T.dot(y))
        SSE = np.sum((y - x.dot(beta))**2)
        SST = np.sum((y - y.mean())**2)
        F_ = np.array(F_)
        F_ = (F_ - SSE) / SSE * (x.shape[0] - x.shape[1])
        p = (stats.f.sf(F_,1,x.shape[0]-x.shape[1]))
        sigma2 = SSE / (x.shape[0]-x.shape[1])
        c = np.linalg.inv(x.T.dot(x)).diagonal()
        t = beta / (c*sigma2)**0.5
        return beta,t, F_,p

In [176]:
model = gradual_regression(df,0.05)

# Task 1

Regression y on x2-x6

In [177]:
init_set = list(df.keys())
y_str = init_set.pop()
beta,t,F,p = model._regression(model._data(init_set),model._data(y_str))
print(model.pretty_report(init_set,beta,t,F,p,'OLS'))


regression result

method: OLS

-----------------

y | beta | t | F_i | p(F)

cons_ | 5922.83 | 2.37 | 0.00 | 0.00
x2 | 4.86 | 1.94 | 3.76 | 0.08
x3 | 2.37 | 2.82 | 7.94 | 0.02
x4 | -817.90 | -4.37 | 19.07 | 0.00
x5 | 14.54 | 0.10 | 0.01 | 0.92
x6 | -846.87 | -2.90 | 8.43 | 0.02




# Task 2

Select proper variable from x2-x6 using backward method.


In [178]:
print(model.backward_reg())

No. 0 ['x2', 'x3', 'x4', 'x5', 'x6']
No. 1 ['x2', 'x3', 'x4', 'x6']

regression result

method: backward

-----------------

y | beta | t | F_i | p(F)

cons_ | 6007.32 | 2.68 | 0.00 | 0.00
x2 | 5.07 | 3.73 | 13.89 | 0.00
x3 | 2.31 | 4.75 | 22.56 | 0.00
x4 | -824.26 | -4.91 | 24.14 | 0.00
x6 | -862.70 | -3.71 | 13.77 | 0.00




# Task 3

Select proper variable from x2-x6 using gradually selecting method.

In [179]:
print(model.gradual_reg())

No. 0 []
No. 1 ['x6']
No. 2 []
No. 3 ['x5']
No. 4 []
No. 5 ['x4']
No. 6 []
No. 7 ['x3']
No. 8 ['x3', 'x4']
No. 9 ['x3']
No. 10 ['x3', 'x5']
No. 11 ['x3', 'x5', 'x4']
No. 12 ['x3', 'x5', 'x4', 'x6']
No. 13 ['x3', 'x5', 'x4']
No. 14 ['x3', 'x5', 'x4', 'x2']
No. 15 ['x3', 'x5', 'x4']

regression result

method: gradual

-----------------

y | beta | t | F_i | p(F)

cons_ | 1412.81 | 0.76 | 0.00 | 0.00
x3 | 3.44 | 4.40 | 19.34 | 0.00
x5 | 348.73 | 3.78 | 14.30 | 0.00
x4 | -415.14 | -2.45 | 6.02 | 0.03




# Task 4

Analyse the difference on above methods.

Different method take different precoss to select variable, but they have a different preference on the number of final model variable.

As for the gradual method, Most important varible will be taken to the model first, and there seems to be more competition between highly-relevant variable, so it is a little bit like winner takes all. In gerenal, it may lose some information of intermediate variable. In this content, x2,x3,x6 may have similiar increasing trend, once the x3 is selected, x2,x6 will have less chance to be selected.

As for the backward method, it build full model at first, so contribution of each in the full environment seems to be more reasonable. And its model have a preference to take in more variable. The disadvantages of it is that it may take more redundant info and cause a wide interval of the estimates. In this context, it is more reasonable and explaintory.